# Route Generating Heuristic

- Trigger Customers (Red, R): Need product immediately within the planning horizon.
- Balance Customers (Yellow, Y): Will not reach safety stock levels within the planning but have a capacity to accept product. They will reach safety stock levels within Tj days after the planning horizon.
- Dump Customers (Black Open Circle, D): Do not need product but are able to receive the remainder of the truck


# Setting the stage
- Import pandas so the data can be stored and manipulated as a dataframe.
- Import geopy to calculate distance between points
- Input the capacity of the trucks, this is assumed to be fixed for all trucks in the preliminary code, however there are different truck sizes in the real fleet
- Set the plant location to determine the distances to all other customers
- Create dummy data that will be used to test the heuristic and code.

In [4]:
import pandas as pd
import numpy as np
import itertools
from geopy.distance import geodesic
from itertools import permutations

In [235]:
#Info: Truck Capacity
truck_cap = 1000
plant_loc = {'lat': 42.975430,
        'lon': -78.853010}

In [236]:
#customer "DUMMY" data
data = {'id': [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 
        'name': ['plant', 'UBN', 'Fort Niagara', 'Buffalo Airport', 'Rochester Airport', 'SUNY Geneseo', 'Airport Batavia', 'NYBP', 'Chestnut Ridge', 'Letchworth', 'Niagara Falls', 'Unity Hospital', 'Darien Lake', 'Medina High School', 'Walmart Supercenter', 'NYSDOC Orleans Correctional'], 
        'lat': [42.975430, 43.000809, 43.232170, 42.933113, 43.128381, 42.796200, 43.029587, 43.089470, 42.712783, 42.656061, 43.086254, 43.190889, 42.918620, 43.209555, 43.234085, 43.245553], 
        'lon': [-78.853010, -78.788971, -79.000671, -78.732171, -77.665379, -77.820494, -78.168698, -78.695171, -78.751342, -77.968785, -79.066905, -77.705040, -78.419043, -78.396706, -78.225102, -78.219151], 
        'type': ['plant', 'Trigger', 'Dump', 'Balance', 'Trigger', 'Dump', 'Balance', 'Trigger', 'Balance', 'Dump', 'Balance', 'Balance', 'Trigger', 'Trigger', 'Balance', 'Dump'],
        'color': ['lightgreen', 'red', 'black', 'orange', 'red', 'black', 'orange', 'red', 'orange', 'black', 'orange', 'orange', 'red', 'red', 'orange', 'black'],
        'empt': [0, 600, 1100, 500, 300, 1100, 500, 750, 200, 1100, 400, 300, 1000, 500, 600, 1100],
        'TTE': [0, 3, 100, 48, 6, 100, 48, 1, 48, 100, 48, 48, 5, 6, 48, 100],
        'Radius': [0, 15, 30, 15, 60, 60, 30, 15, 15, 30, 15, 60, 45, 30, 30, 45],
        'MinDelAmt': [0, 100, 300, 250, 150, 100, 300, 250, 200, 100, 250, 150, 300, 200, 200, 150]}

In [237]:
#DataFrame of customer information: 
#ID, Name, lat, lon, Distance from Plant, Projected Emptiness,
#Customer Type (trigger, balance or dump), Time to Empty
#Radius, Min Delivery Amount
customer1_df = pd.DataFrame.from_dict(data)
#customer_df['Neighborhood'] = 0 #Create new column for neighborhood and populate it with 0 to indicate false
#customer_df['Cone'] = 0 #Create new column for cone and populate it with 0 to indicate false
customer_df = customer1_df[1:].reset_index(drop = True)
customer_df

,id,name,lat,lon,type,color,empt,TTE,Radius,MinDelAmt
0,1,UBN,43.000809,-78.788971,Trigger,red,600,3,15,100
1,2,Fort Niagara,43.232170,-79.000671,Dump,black,1100,100,30,300
2,3,Buffalo Airport,42.933113,-78.732171,Balance,orange,500,48,15,250
3,4,Rochester Airport,43.128381,-77.665379,Trigger,red,300,6,60,150
4,5,SUNY Geneseo,42.796200,-77.820494,Dump,black,1100,100,60,100
5,6,Airport Batavia,43.029587,-78.168698,Balance,orange,500,48,30,300
6,7,NYBP,43.089470,-78.695171,Trigger,red,750,1,15,250
7,8,Chestnut Ridge,42.712783,-78.751342,Balance,orange,200,48,15,200
8,9,Letchworth,42.656061,-77.968785,Dump,black,1100,100,30,100
9,10,Niagara Falls,43.086254,-79.066905,Balance,orange,400,48,15,250


# Creating a veroviz map to display the data
This step is not critical to making the heuristic work, but it is great to visualize the map of customers.
- import veroviz and initialize a nodes data frame
    - Installation instructions can be found at https://veroviz.org/documentation.html
- populate the nodes dataframe with data needed to generate a veroviz map
- initialize the veroviz map

In [238]:
#Create map of customers
import veroviz as vrv
nodes = vrv.initDataframe('nodes')

In [239]:
# Let's go ahead and re-initialize an empty dataframe within this cell:
nodes = vrv.initDataframe('nodes')

# Now, copy the relevant columns from our Station Info dataframe:
# NOTE: We were getting some size mis-match errors until we copied 
#       just a single column first.  
nodes['id'] = customer1_df['id'].values
nodes[['id', 'lat', 'lon', 'nodeName']] = customer1_df[['id', 'lat', 'lon', 'name']].values
nodes[['leafletIconText', 'leafletColor']] = customer1_df[['name', 'color']].values

# Finally, we'll fill in the rest of our nodes dataframe with some hard-coded/constant values:
nodes.loc[:,'altMeters'] = 0
nodes.loc[:,['nodeType', 'leafletIconPrefix', 'leafletIconType']] = [
             'Customer',  'fa',                'star']

In [240]:
nodes

,id,lat,lon,altMeters,nodeName,nodeType,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText
0,0,42.975430,-78.853010,0,plant,Customer,fa,star,lightgreen,plant,NaN,NaN,NaN
1,1,43.000809,-78.788971,0,UBN,Customer,fa,star,red,UBN,NaN,NaN,NaN
2,2,43.232170,-79.000671,0,Fort Niagara,Customer,fa,star,black,Fort Niagara,NaN,NaN,NaN
3,3,42.933113,-78.732171,0,Buffalo Airport,Customer,fa,star,orange,Buffalo Airport,NaN,NaN,NaN
4,4,43.128381,-77.665379,0,Rochester Airport,Customer,fa,star,red,Rochester Airport,NaN,NaN,NaN
5,5,42.796200,-77.820494,0,SUNY Geneseo,Customer,fa,star,black,SUNY Geneseo,NaN,NaN,NaN
6,6,43.029587,-78.168698,0,Airport Batavia,Customer,fa,star,orange,Airport Batavia,NaN,NaN,NaN
7,7,43.089470,-78.695171,0,NYBP,Customer,fa,star,red,NYBP,NaN,NaN,NaN
8,8,42.712783,-78.751342,0,Chestnut Ridge,Customer,fa,star,orange,Chestnut Ridge,NaN,NaN,NaN
9,9,42.656061,-77.968785,0,Letchworth,Customer,fa,star,black,Letchworth,NaN,NaN,NaN


In [241]:
vrv.createLeaflet(nodes=nodes)

# Preparing the data for analysis
- Create a special dataframe consisting of only the trigger customers
- Order them based on their time to emptiness

In [242]:
#DataFrame of trigger customers
trigger_df = pd.DataFrame(customer_df[customer_df['type'] == 'Trigger']).reset_index(drop = True)
trigger_df

,id,name,lat,lon,type,color,empt,TTE,Radius,MinDelAmt
0,1,UBN,43.000809,-78.788971,Trigger,red,600,3,15,100
1,4,Rochester Airport,43.128381,-77.665379,Trigger,red,300,6,60,150
2,7,NYBP,43.089470,-78.695171,Trigger,red,750,1,15,250
3,12,Darien Lake,42.918620,-78.419043,Trigger,red,1000,5,45,300
4,13,Medina High School,43.209555,-78.396706,Trigger,red,500,6,30,200


In [243]:
#DataFrame of trigger customers
dump_df = pd.DataFrame(customer_df[customer_df['type'] == 'Dump']).reset_index(drop = True)
dump_df
Dump = dump_df['id'].tolist()
Dump

[2, 5, 9, 15]

In [244]:
#Sort Trigger Customers by their time to emptiness and this will be the prioritized ordering for how to assign routes
trigger_df.sort_values(by=['TTE']).reset_index(drop = True)

,id,name,lat,lon,type,color,empt,TTE,Radius,MinDelAmt
0,7,NYBP,43.089470,-78.695171,Trigger,red,750,1,15,250
1,1,UBN,43.000809,-78.788971,Trigger,red,600,3,15,100
2,12,Darien Lake,42.918620,-78.419043,Trigger,red,1000,5,45,300
3,4,Rochester Airport,43.128381,-77.665379,Trigger,red,300,6,60,150
4,13,Medina High School,43.209555,-78.396706,Trigger,red,500,6,30,200


# Create the Neighborhood

- Neighborhood needs to be generated for each trigger customer


In [245]:
# Create a list of all of the route trigger id numbers
Route_Triggers = []
Route_Triggers = trigger_df['id'].tolist()
Route_Triggers    

[1, 4, 7, 12, 13]

### Create the neighborhood for each trigger and determine which customers are in that neighborhood. 

    - Create a dictionary that contains the information of all of the customers who are in trigger i's neighborhood.

In [246]:
#Create a dictionary. Keys will be trigger customers, values will be a list of neighbor customers

#list of Keys is the list Route_Triggers, defined above

#Initialize Dictionary
Trig_Neighbors = {}

# iterating through the elements of list "Route_Triggers" initializes 
# all the trigger customers as the keys of dictionary "Trig_Neighbors"
for i in Route_Triggers: 
    Trig_Neighbors[i] = []     
#print(Trig_Neighbors) 

#Save the coordinates of the trigger
for i, row in trigger_df.iterrows():
    #Initialize an empty neighbor list for each trigger
    Neighbor = []
    Trigger_id = trigger_df['id'][i]
    Trigger_lat = trigger_df['lat'][i]
    Trigger_lon = trigger_df['lon'][i]
    radius = trigger_df['Radius'][i]
    loc = (Trigger_lat, Trigger_lon)
    #print(loc)

            
    Tlist = []
    Blist = []
    Dlist = []
    #Save the coordinates of the customer
    for j in range(len(customer_df)-1):
        cust_id = customer_df['id'][j]
        Customer_lat = customer_df['lat'][j]
        Customer_lon = customer_df['lon'][j]
        Customer_Type = customer_df['type'][j]
        cust_loc = (Customer_lat, Customer_lon)
        #print(cust_loc)
        
        #Calculate the distance between the trigger and all other customers to the nearest mile
        #https://pypi.org/project/geopy/
       
        distance = round(geodesic(loc, cust_loc).miles)
        #print (Trigger_id, radius, cust_id, distance)

        #For each customer, compare its distance to the trigger radius
        if ((distance <= radius) & (distance > 0)):
            if Customer_Type == 'Trigger':
                #list of triggers
                Tlist.append(cust_id)
            if Customer_Type == 'Balance':
                #list of balance
                Blist.append(cust_id)
            if Customer_Type == 'Dump':
                #list of dumps
                Dlist.append(cust_id)
            #print(cust_id, Customer_Type)
            #Add neighbor id to the Neighbor list
            #Neighbor.append(cust_id)
            #print(Trigger_id, Tlist, Blist, Dlist)
        '''
        if "conditions for cone" 
            if Customer_Type == 'Trigger':
                #list of triggers
                Tlist.append(cust_id)
            if Customer_Type == 'Balance':
                #list of balance
                Blist.append(cust_id)
            if Customer_Type == 'Dump':
                #list of dumps
                Dlist.append(cust_id)
        '''
    #append lists to Trig_Neighbors
    Trig_Neighbors.setdefault(Trigger_id, []).append(Tlist)
    Trig_Neighbors.setdefault(Trigger_id, []).append(Blist)        
    Trig_Neighbors.setdefault(Trigger_id, []).append(Dlist)
print (Trig_Neighbors)

#Documentation of Trig_Neighbors
# Keys: The trigger customers
# List 1: The trigger customers in the neighborhood
# List 2: The balance customers in the neighborhood
# List 3: The dump customers in the neighborhood

{1: [[7], [3, 10], []], 4: [[1, 7, 12, 13], [3, 6, 11, 14], [5, 9]], 7: [[1], [3], []], 12: [[1, 4, 7, 13], [3, 6, 8, 10, 11, 14], [2, 5, 9]], 13: [[1, 7, 12], [3, 6, 14], []]}


# Create the Cone

- Customers who are located in the cone extending between the trigger neighborhood and the plant can also be considered for the route


In [320]:
#import shapely
from shapely.geometry import Point, Polygon

# Create Point objects
p1 = Point(24.952242, 60.1696017)
p2 = Point(24.976567, 60.1612500)

# Create a Polygon
coords = [(24.950899, 60.169158), (24.953492, 60.169158), (24.953510, 60.170104), (24.950958, 60.169990)]
poly = Polygon(coords)

ModuleNotFoundError: No module named 'shapely'

# Begin Generating Routes

- Store Routes as lists? Then Store as ordered tuples after the TSP has been solved.
    - What is the structure to have them stored as within the list as a grouping [1-2-3]

- How can I store partial routes as fixed so that the 3rd/4th stops can vary?
- Be sure to store the total distance of the routes after the TSP has been solved

Logic: For each trigger, create a list of lists for partial routes
    ['amount left in truck' , [trigger, cust 2]]
    

In [247]:
Route_Triggers

[1, 4, 7, 12, 13]

In [248]:
trigger_df

,id,name,lat,lon,type,color,empt,TTE,Radius,MinDelAmt
0,1,UBN,43.000809,-78.788971,Trigger,red,600,3,15,100
1,4,Rochester Airport,43.128381,-77.665379,Trigger,red,300,6,60,150
2,7,NYBP,43.089470,-78.695171,Trigger,red,750,1,15,250
3,12,Darien Lake,42.918620,-78.419043,Trigger,red,1000,5,45,300
4,13,Medina High School,43.209555,-78.396706,Trigger,red,500,6,30,200


In [249]:
Trig_Neighbors

{1: [[7], [3, 10], []],
 4: [[1, 7, 12, 13], [3, 6, 11, 14], [5, 9]],
 7: [[1], [3], []],
 12: [[1, 4, 7, 13], [3, 6, 8, 10, 11, 14], [2, 5, 9]],
 13: [[1, 7, 12], [3, 6, 14], []]}

In [250]:
customer_df

,id,name,lat,lon,type,color,empt,TTE,Radius,MinDelAmt
0,1,UBN,43.000809,-78.788971,Trigger,red,600,3,15,100
1,2,Fort Niagara,43.232170,-79.000671,Dump,black,1100,100,30,300
2,3,Buffalo Airport,42.933113,-78.732171,Balance,orange,500,48,15,250
3,4,Rochester Airport,43.128381,-77.665379,Trigger,red,300,6,60,150
4,5,SUNY Geneseo,42.796200,-77.820494,Dump,black,1100,100,60,100
5,6,Airport Batavia,43.029587,-78.168698,Balance,orange,500,48,30,300
6,7,NYBP,43.089470,-78.695171,Trigger,red,750,1,15,250
7,8,Chestnut Ridge,42.712783,-78.751342,Balance,orange,200,48,15,200
8,9,Letchworth,42.656061,-77.968785,Dump,black,1100,100,30,100
9,10,Niagara Falls,43.086254,-79.066905,Balance,orange,400,48,15,250


# Create the Routes and Partial Routes 
For each trigger, sort through their neighbors. Create groupings of triggers with neighbors to begin filling potential routes.
- If the emptiness of the trigger itself requires a full truck load of product, then the trigger is added to the list of complete routes.
- If the emptiness of the trigger plus the emptiness of its neighbor is less than a full truck load, the pair is added to the Partial Routes list.
- If the emptiness of the trigger plus the emptiness of its neighbors is more than a full truck load, and the neighbor's minimum delivery amount is fulfilled, then the pair is added to the Complete Routes list.

In [251]:
partial_routes = []
complete_routes = []
for i in range(len(Route_Triggers)):
    truck_cap = 1000
    for key in Trig_Neighbors:
        if Route_Triggers[i] == key:
            #print ("This is a new i", i, key, Trig_Neighbors[key])
            for j in Trig_Neighbors[key]: #the first j is other triggers, second balance, third dump
                #print("This is the new j", j)
                for k in j: # at this point we are iterating through each neighbor associated with the trigger customer
                    #print("This is the new k", k)
                    truck_cap = 1000
                    truck_cap = truck_cap - customer_df.iloc[key-1]['empt']
                    #print(truck_cap)
                    
                    #trigger needs a full truck
                    if truck_cap < 0.1: 
                        if [key] not in complete_routes:
                            complete_routes.append([key])
                            #print("COMPLETED ROUTES", complete_routes)
                    
                    #trigger + neighbor takes a partial truck
                    elif truck_cap > 0.1 and truck_cap - customer_df.iloc[k-1]['empt'] > 0.1:
                        #print("k", k, "emptiness", customer_df.iloc[k-1]['empt'])
                        #add a second customer but still doesn't complete the truck
                        #for truck_cap - customer_df.iloc[k]['empt'] > 0.1:
                        if [key,k] not in partial_routes:
                            partial_routes.append([key,k])
                            #print("PARTIAL ROUTES", partial_routes)
                        
                    #trigger + neighbor takes the rest of the truck
                    #if the truck capacity - potential customer emptiness is negative but the min delivery amount is satisfied then the route is complete
                    elif truck_cap > 0.1 and truck_cap - customer_df.iloc[k-1]['empt'] < 0.1:
                        if truck_cap > customer_df.iloc[k-1]['MinDelAmt']:
                            if [key,k] not in complete_routes:
                                complete_routes.append([key,k])
                                #print("COMPLETED ROUTES", complete_routes)
print("COMPLETED ROUTES", complete_routes)     
print("PARTIAL ROUTES", partial_routes)

COMPLETED ROUTES [[1, 7], [1, 3], [1, 10], [4, 7], [4, 12], [4, 5], [4, 9], [7, 1], [12], [13, 1], [13, 7], [13, 12], [13, 3], [13, 6], [13, 14]]
PARTIAL ROUTES [[4, 1], [4, 13], [4, 3], [4, 6], [4, 11], [4, 14]]


# Complete the Partial Routes 
Iterate through the trigger's neighbors and find a third customer within the neighborhood that can accept the product remaining in the truck.

In [252]:
#Trig_Neighbors

In [255]:
#iterate through the partial routes and create full routes
print("------------------------------------------")
print("PARTIAL ROUTES BEFORE:", partial_routes)
print('COMPLETE ROUTES BEFORE:', complete_routes)
print('------------------------------------------')
#ADD WHILE LOOP, WHILE PARTIAL ROUTES IS NOT EMPTY, CONTINUE TO ITERATE THROUGH. ALL ROUTES MUST BE COMPLETE.
#while len(partial_routes) > 0:
for i in partial_routes:
    print('------------------------------------------')
    print("NEW PARTIAL ROUTE EXAMINED", i)
    print('------------------------------------------')
    #find the amount left in the truck after the first two customers on the partial route
    truck_cap = 1000

    #recalculate the remaining cpacity in the truck by subtracting the emptiness of partial route customers
    for j in i:
        #print("j", j, "emptiness of j", customer_df.iloc[j-1]['empt'])
        truck_cap = truck_cap - customer_df.iloc[j-1]['empt']
        #print("truck_cap", truck_cap)
    #print("---------find trigger's potential neighbors for a third customer---------")

    #if the key is equal to the first element of "partial_routes", 
    #then search that trigger for the remaining neighbors as potential customers
    neighbor_i = []
    for key in Trig_Neighbors:
        if key == i[0]:
            #print (Trig_Neighbors.get(key, "none"))
            neighbor_i = Trig_Neighbors.get(key, "none")
            #print("neighbor_i", neighbor_i)

   
    for m in neighbor_i:
        #print("m", m)
        for n in m:
            #print("n", n, "i", i)
            #print("Complete Routes", complete_routes)
            #print("Partial Routes", partial_routes)
            
            if n not in i: #this neighbor can be added to the partial route
                #print('-----Check Customer n-----')
                #print("i", i, "n", n, "neighbor emptiness", customer_df.iloc[n-1]['empt'])
                a = []

                #trigger + 2nd neighbor takes a partial truck
                if truck_cap > 0.1 and truck_cap - customer_df.iloc[n-1]['empt'] > 0.1:
                    #add a third customer but still doesn't complete the truck
                    #print('i', i)
                    #i.append(n)
                    for each in i:
                        #print("Each of i", each)
                        a.append(each)
                        
                    if [a, n] not in partial_routes:
                        a.append(n)
                        #print('a', a)
                        partial_routes.append(a)
                        #print("PARTIAL3 ROUTES", partial_routes)
                    #print('New i', i, "Revert to old i")
                    #print('Back to partial route i')


                #trigger + 2nd neighbor takes the rest of the truck
                #if the truck capacity - potential customer emptiness is negative but the min delivery amount is satisfied then the route is complete
                elif truck_cap > 0.1 and truck_cap - customer_df.iloc[n-1]['empt'] < 0.1:
                    if truck_cap >= customer_df.iloc[n-1]['MinDelAmt']:
                        #print('i', i)
                        #i.append(n)
                       # print("***New i***", i)
                        for each in i:
                            #print("Each in i", each, "a", a)
                            a.append(each)
                            #print("a", a)
                        if [a,n] not in complete_routes:
                            a.append(n)
                            #print('a', a)
                            complete_routes.append(a)
                            #print("COMPLETED ROUTES", complete_routes)
                            #print('New i', i, "Revert to old i")
                        #print('Back to partial route i')
                        #print("Complete Routes", complete_routes)
                            
                    #else:
                     #   print("Due to min delivery amount, Skip Customer", n)       
                            
                
            #else:
             #   print("Due to n being in i, Skip Customer", n)

    partial_routes.remove(i)
    print('i removed from partial routes:', i)
    print('PARTIAL ROUTES AFTER', partial_routes)
    print("Complete Routes After", complete_routes)

------------------------------------------
PARTIAL ROUTES BEFORE: [[4, 6]]
COMPLETE ROUTES BEFORE: [[1, 7], [1, 3], [1, 10], [4, 7], [4, 12], [4, 5], [4, 9], [7, 1], [12], [13, 1], [13, 7], [13, 12], [13, 3], [13, 6], [13, 14], [4, 1, 5], [4, 1, 9], [4, 3, 1], [4, 3, 13], [4, 3, 11], [4, 3, 14], [4, 3, 5], [4, 3, 9], [4, 11, 1], [4, 11, 7], [4, 11, 12], [4, 11, 13], [4, 11, 3], [4, 11, 6], [4, 11, 14], [4, 11, 5], [4, 11, 9], [4, 13, 1], [4, 13, 11], [4, 13, 14], [4, 13, 5], [4, 13, 9], [4, 14, 1], [4, 14, 5], [4, 14, 9]]
------------------------------------------
------------------------------------------
NEW PARTIAL ROUTE EXAMINED [4, 6]
------------------------------------------
i removed from partial routes: [4, 6]
PARTIAL ROUTES AFTER []
Complete Routes After [[1, 7], [1, 3], [1, 10], [4, 7], [4, 12], [4, 5], [4, 9], [7, 1], [12], [13, 1], [13, 7], [13, 12], [13, 3], [13, 6], [13, 14], [4, 1, 5], [4, 1, 9], [4, 3, 1], [4, 3, 13], [4, 3, 11], [4, 3, 14], [4, 3, 5], [4, 3, 9], [4, 1

In [ ]:
#Separate back out by trigger again
#first element = key
#add to dictionary {key: [route], [route], [route]}

In [265]:
potential_routes = {}
for i in Route_Triggers: 
    potential_routes[i] = []

potential_routes

{1: [], 4: [], 7: [], 12: [], 13: []}

In [266]:
for key in potential_routes:
    #key_routes = []
    for i in complete_routes:
        if i[0] == key:
            #print(i)
            #key_routes.append(i)
            #print(key_routes)
            potential_routes.setdefault(key, ).append(i) 

print(potential_routes)

{1: [[1, 7], [1, 3], [1, 10]], 4: [[4, 7], [4, 12], [4, 5], [4, 9], [4, 1, 5], [4, 1, 9], [4, 3, 1], [4, 3, 13], [4, 3, 11], [4, 3, 14], [4, 3, 5], [4, 3, 9], [4, 11, 1], [4, 11, 7], [4, 11, 12], [4, 11, 13], [4, 11, 3], [4, 11, 6], [4, 11, 14], [4, 11, 5], [4, 11, 9], [4, 13, 1], [4, 13, 11], [4, 13, 14], [4, 13, 5], [4, 13, 9], [4, 14, 1], [4, 14, 5], [4, 14, 9], [4, 6, 1], [4, 6, 13], [4, 6, 11], [4, 6, 14], [4, 6, 5], [4, 6, 9]], 7: [[7, 1]], 12: [[12]], 13: [[13, 1], [13, 7], [13, 12], [13, 3], [13, 6], [13, 14]]}


## Solve TSP for each route to find the order and the total distance

In [ ]:
#for all potential routes, solve the TSP
#save output as {Trigger: [ordered list] , total distance}

In [258]:
customer1_df = pd.DataFrame.from_dict(data)
customer1_df

,id,name,lat,lon,type,color,empt,TTE,Radius,MinDelAmt
0,0,plant,42.975430,-78.853010,plant,lightgreen,0,0,0,0
1,1,UBN,43.000809,-78.788971,Trigger,red,600,3,15,100
2,2,Fort Niagara,43.232170,-79.000671,Dump,black,1100,100,30,300
3,3,Buffalo Airport,42.933113,-78.732171,Balance,orange,500,48,15,250
4,4,Rochester Airport,43.128381,-77.665379,Trigger,red,300,6,60,150
5,5,SUNY Geneseo,42.796200,-77.820494,Dump,black,1100,100,60,100
6,6,Airport Batavia,43.029587,-78.168698,Balance,orange,500,48,30,300
7,7,NYBP,43.089470,-78.695171,Trigger,red,750,1,15,250
8,8,Chestnut Ridge,42.712783,-78.751342,Balance,orange,200,48,15,200
9,9,Letchworth,42.656061,-77.968785,Dump,black,1100,100,30,100


In [259]:
#distance calculations... add a total distance matrix
distance_M = []
for i in range(len(customer1_df)):
    A = []
    #print(i, customer1_df['id'][i], customer1_df['lat'][i], customer1_df['lon'][i])
    #Find coordinates of the first customer
    Loc1_id = customer1_df['id'][i]
    Loc1_lat = customer1_df['lat'][i]
    Loc1_lon = customer1_df['lon'][i]
    Loc1 = (Loc1_lat, Loc1_lon)
    
        #Save the coordinates of the second customer
    for j in range(len(customer1_df)):
        #print (i, j, Loc1_id, customer1_df['id'][j])
        Loc2_id = customer1_df['id'][j]
        Loc2_lat = customer1_df['lat'][j]
        Loc2_lon = customer1_df['lon'][j]
        Loc2 = (Loc2_lat, Loc2_lon)


        #Calculate the distance between the customers to the nearest mile
        #https://pypi.org/project/geopy/

        distance = round(geodesic(Loc1, Loc2).miles)
        A.append(distance)
        #print (Loc1_id, Loc2_id, distance)
        #print(A)
    distance_M.append(A)
print("Distance M", distance_M)

Distance M [[0, 4, 19, 7, 61, 54, 35, 11, 19, 50, 13, 60, 22, 28, 36, 37], [4, 0, 19, 5, 58, 51, 31, 8, 20, 48, 15, 56, 20, 25, 33, 33], [19, 19, 0, 25, 68, 67, 44, 18, 38, 66, 11, 65, 37, 31, 39, 39], [7, 5, 25, 0, 56, 47, 29, 11, 15, 43, 20, 55, 16, 26, 33, 34], [61, 58, 68, 56, 0, 24, 26, 52, 62, 36, 71, 5, 41, 37, 29, 29], [54, 51, 67, 47, 24, 0, 24, 49, 48, 12, 66, 28, 32, 41, 37, 37], [35, 31, 44, 29, 26, 24, 0, 27, 37, 28, 46, 26, 15, 17, 14, 15], [11, 8, 18, 11, 52, 49, 27, 0, 26, 47, 19, 51, 18, 17, 26, 26], [19, 20, 38, 15, 62, 48, 37, 26, 0, 40, 30, 62, 22, 39, 45, 46], [50, 48, 66, 43, 36, 12, 28, 47, 40, 0, 63, 39, 29, 44, 42, 43], [13, 15, 11, 20, 71, 66, 46, 19, 30, 63, 0, 69, 35, 35, 44, 44], [60, 56, 65, 55, 5, 28, 26, 51, 62, 39, 69, 0, 41, 35, 26, 26], [22, 20, 37, 16, 41, 32, 15, 18, 22, 29, 35, 41, 0, 20, 24, 25], [28, 25, 31, 26, 37, 41, 17, 17, 39, 44, 35, 35, 20, 0, 9, 9], [36, 33, 39, 33, 29, 37, 14, 26, 45, 42, 44, 26, 24, 9, 0, 1], [37, 33, 39, 34, 29, 37, 15

In [313]:
route_distances_dict = {}
for i in Route_Triggers: 
    route_distances_dict[i] = []

for key in potential_routes:
    #print("-------New Key-------")
    for element in potential_routes[key]:
        #print("-------New Element-------")
        trip = []
        check_dist = []
        
        #Routes with one customer: COMPLETED
        if len(element) == 1:
            route = element
            #print(route)
            
            route_distance = 0
            
            for i in element:
                #print("-------New Customer-------")
                route_distance = route_distance + 2*distance_M[0][i]
                
                trip.append(route)
                trip.append(route_distance)
                #print('trip', trip)
                route_distances_dict.setdefault(key, ).append(trip)
                #print(route_distances_dict)
 
        
        
        #Routes with 2 customers
        if len(element) == 2:
            route = element
            #print(route)
            
            route_distance = 0
            first = True
            second = True
            for i in element:
                #print("-------New Customer-------")
                if first:
                    a = i
                    #print(a, i, distance_M[0][a])
                    route_distance = route_distance + distance_M[0][a]
                    #print(route_distance)
                    first = False
                
                
                elif second:
                    b = i
                    #print(a, b, i, distance_M[a][b], distance_M[b][0])
                    route_distance = route_distance + distance_M[a][b] + distance_M[b][0]
                    second = False
                    
                    trip.append(route)
                    trip.append(route_distance)
                    #print('trip', trip)
                    route_distances_dict.setdefault(key, ).append(trip)
                    #print(route_distances_dict)

        #Routes with THREE OR FOUR customers:
        if len(element) >= 3:
            perm = [] #create a list of permutations, we will iterate over all to see which are feasbile, and which is optimal
            cut = []
            cut_feas = []
            route = element
            #print(route)
            perm = list(itertools.permutations(route, len(element)))
            #print(perm)
            
            #search the list of dump customers, if a dump customer is in the route, that customer must be in position 2 if length is 3,
            #or position 3 if length is 4.
            for i in Dump:
                #print("dump", i)
                if i in route:
                    #print(route, i)
                    
                    #create a list of elements that need to be cut 
                    #(needs to be done this way and not by perm.remove or else the ordering of j will cause some elements to be skipped after the list is changed)
                    for j in perm: 
                        #print(j)
                        if j[-1] != i:
                            #print(j)
                            cut.append(j)
                            
                        
                    for each in cut: #actually cut the elements from perm
                        perm.remove(each)
                    #print(perm)
                    
            if len(element) == 3:
                for j in perm:
                    #print("perm", perm, 'j', j)
                    #print("-------New Permutation-------")
                    route_distance = 0
                    first = True
                    second = True
                    third = True
                    for i in j:
                        if first:
                            a = i
                            #print(a, i, distance_M[0][a])
                            route_distance = route_distance + distance_M[0][a]
                            #print(route_distance)
                            first = False


                        elif second:
                            b = i
                            #print(a, b, i, distance_M[a][b], distance_M[b][0])
                            route_distance = route_distance + distance_M[a][b]
                            second = False

                        elif third:
                            c = i
                            #print(a, b, i, distance_M[a][b], distance_M[b][0])
                            route_distance = route_distance + distance_M[b][c] + distance_M[c][0]
                            third = False

                            #check_dist.append(route)
                            check_dist.append(route_distance)
                            #print(check_dist)
            
            if len(element) == 4:
                for j in perm:
                    #print("perm", perm, 'j', j)
                    #print("-------New Permutation-------")
                    route_distance = 0
                    first = True
                    second = True
                    third = True
                    last = True
                    for i in j:
                        if first:
                            a = i
                            route_distance = route_distance + distance_M[0][a]
                            #print(route_distance)
                            first = False


                        elif second:
                            b = i
                            route_distance = route_distance + distance_M[a][b]
                            second = False

                        elif third:
                            c = i
                            route_distance = route_distance + distance_M[b][c]
                            third = False

                        elif last:
                            d = i
                            route_distance = route_distance + distance_M[c][d] + distance_M[d][0]
                            last = False
                            #check_dist.append(route)
                            check_dist.append(route_distance)
                            
            #print("----------------------------")        
            #print("-------FIND MIN ROUTE-------")
            #print("----------------------------")
            #print("-------Check min fill-------")
            #print('routes being considered', perm)
            
            for i in perm:
                #print("----------------------------")
                #print('check new route in perm')
                #print("----------------------------")
                truck_cap = 1000
                #print('i', i)
                pos = perm.index(i)
                #print(pos)
                #print('pos', pos, 'check_dist', check_dist[pos])
                first = True
                ''''''
                for j in i:
                    if first:
                        #print('first', j, truck_cap, customer_df.iloc[j-1]['MinDelAmt'], customer_df.iloc[j-1]['empt'])
                        truck_cap = truck_cap - customer_df.iloc[j-1]['empt']
                        first = False
                    
                    elif truck_cap < customer_df.iloc[j-1]['MinDelAmt']: 
                        #print("INFEASIBLE ROUTE", j, 'truck_cap', truck_cap, customer_df.iloc[j-1]['MinDelAmt'])
                        pos = perm.index(i)
                        #print('pos', pos, 'check_dist', check_dist[pos], 'cut_feas', cut_feas, 'cut_dist', cut_dist)
                        cut_feas.append(perm[pos])
                        #cut_dist.append(pos)
                        #perm.remove(perm[pos])
                        #check_dist.remove(check_dist[pos])
                        #print('perm', perm, 'check_dist', check_dist, 'cut_feas', cut_feas, 'cut_dist', cut_dist)
                        break
                        
                    
                    elif truck_cap >= customer_df.iloc[j-1]['MinDelAmt']:
                        #print('next', j, truck_cap, customer_df.iloc[j-1]['MinDelAmt'], customer_df.iloc[j-1]['empt'])
                        truck_cap = truck_cap - customer_df.iloc[j-1]['empt']
                        
                    else:
                        print("Something went wrong here...")
            
            
            for each in cut_feas: #actually cut the elements from perm
                #print("----------------------------")
                #print("-------remove cut list from perm-------")
                #print("----------------------------")
                #print('perm start', perm, 'dist start', check_dist)
                #print(each)
                pos = perm.index(each)
                #print('pos', pos, 'check_dist', check_dist, check_dist[pos])
                perm.remove(each)
                check_dist.remove(check_dist[pos])
                #print('remaining in perm', perm, 'remaining in dist', check_dist)
                
                        
                    
            #print("-------FIND MIN ROUTE-------")
            #minimum(check_dist, len(check_dist))
            minpos = check_dist.index(min(check_dist))
            #print(minpos)
            trip.append(perm[minpos])
            trip.append(check_dist[minpos])
            #print('trip', trip)
            route_distances_dict.setdefault(key, ).append(trip)    
                 
                
print(route_distances_dict)            

{1: [[[1, 7], 23], [[1, 3], 16], [[1, 10], 32]], 4: [[[4, 7], 124], [[4, 12], 124], [[4, 5], 139], [[4, 9], 147], [(1, 4, 5), 140], [(1, 4, 9), 148], [(4, 3, 1), 125], [(4, 3, 13), 128], [(4, 11, 3), 128], [(4, 3, 14), 128], [(3, 4, 5), 141], [(3, 4, 9), 149], [(11, 4, 1), 126], [(4, 11, 7), 128], [(4, 11, 12), 128], [(4, 11, 13), 129], [(4, 11, 3), 128], [(11, 4, 6), 126], [(11, 4, 14), 128], [(11, 4, 5), 143], [(11, 4, 9), 151], [(4, 13, 1), 127], [(4, 11, 13), 129], [(4, 13, 14), 130], [(13, 4, 5), 143], [(13, 4, 9), 151], [(4, 14, 1), 127], [(14, 4, 5), 143], [(14, 4, 9), 151], [(6, 4, 1), 122], [(4, 6, 13), 126], [(6, 4, 11), 126], [(4, 6, 14), 126], [(6, 4, 5), 139], [(6, 4, 9), 147]], 7: [[[7, 1], 23]], 12: [[[12], 44]], 13: [[[13, 1], 57], [[13, 7], 56], [[13, 12], 70], [[13, 3], 61], [[13, 6], 80], [[13, 14], 73]]}


In [314]:
#Tester Section

'''
route_distances_dict = {}
for i in Route_Triggers: 
    route_distances_dict[i] = []

for key in potential_routes:
    print("-------New Key-------")
    for element in potential_routes[key]:
        print("-------New Element-------")
        trip = []
        check_dist = []
        check_fills = []

        #Routes with THREE OR FOUR customers:
        if len(element) >= 3:
            perm = [] #create a list of permutations, we will iterate over all to see which are feasbile, and which is optimal
            cut = []
            cut_feas = []
            #cut_dist = []
            route = element
            print(route)
            perm = list(itertools.permutations(route, len(element)))
            print('permutations of the route', perm)
            
            #search the list of dump customers, if a dump customer is in the route, that customer must be in position 2 if length is 3,
            #or position 3 if length is 4.
            for i in Dump:
                print("-------Check for dump customers in the permutations-------")
                print("dump", i)
                if i in route:
                    print("-------There is a dump on the route-------")
                    print(route, i)
                    
                    #create a list of elements that need to be cut 
                    #(needs to be done this way and not by perm.remove or else the ordering of j will cause some elements to be skipped after the list is changed)
                    for j in perm: 
                        #print(j)
                        print("-------remove permutations where dump is not last, add to cut list-------")
                        if j[-1] != i:
                            #print(j)
                            cut.append(j)
                            
                        
                    for each in cut: #actually cut the elements from perm
                        print("-------remove cut list from perm-------")
                        perm.remove(each)
                    print('remaining in perm', perm)
                    
            if len(element) == 3:
                for j in perm:
                    print("perm", perm, 'j', j)
                    print("-------New Permutation Distance Calculated-------")
                    route_distance = 0
                    first = True
                    second = True
                    third = True
                    for i in j:
                        if first:
                            a = i
                            print('first element', a, i, distance_M[0][a])
                            route_distance = route_distance + distance_M[0][a]
                            print(route_distance)
                            first = False


                        elif second:
                            b = i
                            print('second element', a, b, i, distance_M[a][b])
                            route_distance = route_distance + distance_M[a][b]
                            second = False

                        elif third:
                            c = i
                            print('third element', a, b, i, distance_M[b][c], distance_M[c][0])
                            route_distance = route_distance + distance_M[b][c] + distance_M[c][0]
                            third = False

                            #check_fills.append(route)
                            check_dist.append(route_distance)
                            print('distance', check_dist)
            
            if len(element) == 4:
                for j in perm:
                    #print("perm", perm, 'j', j)
                    #print("-------New Permutation-------")
                    route_distance = 0
                    first = True
                    second = True
                    third = True
                    last = True
                    for i in j:
                        if first:
                            a = i
                            #print(a, i, distance_M[0][a])
                            route_distance = route_distance + distance_M[0][a]
                            #print(route_distance)
                            first = False


                        elif second:
                            b = i
                            #print(a, b, i, distance_M[a][b], distance_M[b][0])
                            route_distance = route_distance + distance_M[a][b]
                            second = False

                        elif third:
                            c = i
                            #print(a, b, i, distance_M[a][b], distance_M[b][0])
                            route_distance = route_distance + distance_M[b][c]
                            third = False

                        elif last:
                            d = i
                            #print(a, b, i, distance_M[a][b], distance_M[b][0])
                            route_distance = route_distance + distance_M[c][d] + distance_M[d][0]
                            last = False
                            check_fills.append(route)
                            check_dist.append(route_distance)
                            #print(check_dist)

            print("----------------------------")        
            print("-------FIND MIN ROUTE-------")
            print("----------------------------")
            print("-------Check min fill-------")
            print('routes being considered', perm)
            
            for i in perm:
                print("----------------------------")
                print('check new route in perm')
                print("----------------------------")
                truck_cap = 1000
                print('i', i)
                pos = perm.index(i)
                print(pos)
                print('pos', pos, 'check_dist', check_dist[pos])
                first = True
                ''''''
                for j in i:
                    if first:
                        print('first', j, truck_cap, customer_df.iloc[j-1]['MinDelAmt'], customer_df.iloc[j-1]['empt'])
                        truck_cap = truck_cap - customer_df.iloc[j-1]['empt']
                        first = False
                    
                    elif truck_cap < customer_df.iloc[j-1]['MinDelAmt']: 
                        print("INFEASIBLE ROUTE", j, 'truck_cap', truck_cap, customer_df.iloc[j-1]['MinDelAmt'])
                        pos = perm.index(i)
                        print('pos', pos, 'check_dist', check_dist[pos], 'cut_feas', cut_feas, 'cut_dist', cut_dist)
                        cut_feas.append(perm[pos])
                        #cut_dist.append(pos)
                        #perm.remove(perm[pos])
                        #check_dist.remove(check_dist[pos])
                        print('perm', perm, 'check_dist', check_dist, 'cut_feas', cut_feas, 'cut_dist', cut_dist)
                        break
                        
                    
                    elif truck_cap >= customer_df.iloc[j-1]['MinDelAmt']:
                        print('next', j, truck_cap, customer_df.iloc[j-1]['MinDelAmt'], customer_df.iloc[j-1]['empt'])
                        truck_cap = truck_cap - customer_df.iloc[j-1]['empt']
                        
                    else:
                        print("Something went wrong here...")
            
            a = 0
            for each in cut_feas: #actually cut the elements from perm
                print("----------------------------")
                print("-------remove cut list from perm-------")
                print("----------------------------")
                print('perm start', perm, 'dist start', check_dist)
                print(each)
                pos = perm.index(each)
                print('pos', pos, 'check_dist', check_dist, check_dist[pos])
                perm.remove(each)
                check_dist.remove(check_dist[pos])
                print('remaining in perm', perm, 'remaining in dist', check_dist)
                
                        
                ''''''      
                
            
            
            
            
            
            #minimum(check_dist, len(check_dist))
            print('------Narrow down list to "optimal" Route------')
            print('remaining in perm', perm, 'remaining in dist', check_dist)
            minpos = check_dist.index(min(check_dist))
            print('minpos', minpos)
            trip.append(perm[minpos])
            trip.append(check_dist[minpos])
            print('trip', trip)
            route_distances_dict.setdefault(key, ).append(trip)
print(route_distances_dict)
                    

'''            

'\nroute_distances_dict = {}\nfor i in Route_Triggers: \n    route_distances_dict[i] = []\n\nfor key in potential_routes:\n    print("-------New Key-------")\n    for element in potential_routes[key]:\n        print("-------New Element-------")\n        trip = []\n        check_dist = []\n        check_fills = []\n\n        #Routes with THREE OR FOUR customers:\n        if len(element) >= 3:\n            perm = [] #create a list of permutations, we will iterate over all to see which are feasbile, and which is optimal\n            cut = []\n            cut_feas = []\n            #cut_dist = []\n            route = element\n            print(route)\n            perm = list(itertools.permutations(route, len(element)))\n            print(\'permutations of the route\', perm)\n            \n            #search the list of dump customers, if a dump customer is in the route, that customer must be in position 2 if length is 3,\n            #or position 3 if length is 4.\n            for i in Du

# TSP Solving Questions

Done: - Do I solve the TSP only between the customers in the route, then add the distance from the plant to the first customer, and last customer to the plant?
    - or do I add the plant to it as well and solve a closed loop TSP? If so, how do I do that?
    
- if the two customers take 90%, but no customer can be added to the route, the route is infeasible. Don't allow the truck to come back with more than 10% of product in the truck.

Done: - check all combinations 123, 321, etc. Find the distance for all combinations, find the shortest and check if the min fills are satisfied.

Done: -  how to find the loops to make all combinations for 3 or 4 customers.

- cone. Create a box, and determine if it is in the box. Calculate the diagonal/tangent line equation and determine if it is the "correct" side of the diagonal or not. If both are satisfied, then it is in the cone. (must check the left rectangle and the right rectangle to make sure it is correct). 